In [1]:
import os
os.chdir('/home/xyz/data/haoyan/bandits')

import bandits as bd
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

/home/xyz/miniconda3/envs/bandit/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def f(w, x, y, z):
    if w == 1 and x == 1:
        return y + z
    if w == 2 and x == 1:
        return y + z + 1
    if w == 1 and x == 2:
        return y - z - 1
    if w == 2 and x == 2:
        return y - z
    
def g(w, x, y, z):
    if w == 1 and x == 1:
        return np.sin(y * z) / (y)
    if w == 2 and x == 1:
        return np.cos(y * z) / (y)
    if w == 1 and x == 2:
        return -np.sin(y * z) / (y)
    if w == 2 and x == 2:
        return -np.cos(y * z) / (y)

In [3]:
train = pd.read_csv('./data/train.csv')
features = ['Pclass', 'Sex', 'Age', 'Fare']
label = 'Survived'

train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})

X_train, X_test, y_train, y_test = train_test_split(train[features], train[label], test_size=0.33, random_state=42)

def f(num_leaves, max_depth, learning_rate, n_estimators, reg_lambda):
    model = lgb.LGBMClassifier(
        num_leaves=num_leaves,
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        reg_lambda=reg_lambda,
    )

    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, preds)

In [4]:
bandit = bd.GaussianBandit(
    f,
    params_list_dict={
        'num_leaves': [2, 3, 4],
        'max_depth': [2, 3, 4, 5],
        'n_estimators': [100, 200, 300, 400, 1000],
    },
    pbounds={
        'learning_rate': [0.001, 0.5],
        'reg_lambda': [0.01, 1],
    },
    init_points=2,
    n_iter=10,
)
n_trials = 100

[LightGBM] [Info] Number of positive: 222, number of negative: 374
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167
[LightGBM] [Info] Number of data points in the train set: 596, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.372483 -> initscore=-0.521578
[LightGBM] [Info] Start training from score -0.521578
[LightGBM] [Info] Number of positive: 222, number of negative: 374
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167
[LightGBM] [Info] Number of data points in the train set: 596, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.372483 -> initscore=-0.521578
[LightGBM] [Info] Start training from score -0.521578
[LightGBM] [Info] Number of 

In [5]:
policy = bd.SoftmaxPolicy()
agent = bd.GradientAgent(bandit, policy, alpha=0.5, baseline=False)
env = bd.Environment(bandit, agent, 'Gradient Agents')
scores, actions, optimal = env.run(n_trials)

{'num_leaves': 4, 'max_depth': 4, 'n_estimators': 300}
|   iter    |  target   | learni... | reg_la... |
-------------------------------------------------
[LightGBM] [Info] Number of positive: 222, number of negative: 374
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167
[LightGBM] [Info] Number of data points in the train set: 596, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.372483 -> initscore=-0.521578
[LightGBM] [Info] Start training from score -0.521578
| 1         | 0.8857    | 0.05445   | 0.7185    |
[LightGBM] [Info] Number of positive: 222, number of negative: 374
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000058 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167
[LightGBM] [Info] Number of data points in the 

In [9]:
actions[np.argmax(scores)], scores[np.argmax(scores)]

(((4, 4, 1000),
  {'learning_rate': 0.0034291344554475533, 'reg_lambda': 0.14909366137917407}),
 0.8922142857142857)

In [7]:
scores

array([0.8857381 , 0.8857381 , 0.87288095, 0.89021429, 0.8857381 ,
       0.88733333, 0.87264286, 0.88588095, 0.88733333, 0.88733333,
       0.87404762, 0.89221429, 0.87288095, 0.88233333, 0.8887381 ,
       0.87240476, 0.89121429, 0.8865    , 0.87240476, 0.88233333,
       0.8865    , 0.87240476, 0.89121429, 0.8737381 , 0.87288095,
       0.87240476, 0.88733333, 0.8865    , 0.89121429, 0.8865    ,
       0.87288095, 0.89121429, 0.88733333, 0.89021429, 0.89121429,
       0.88669048, 0.8857381 , 0.88588095, 0.8865    , 0.89121429,
       0.8737381 , 0.87240476, 0.89021429, 0.87288095, 0.87288095,
       0.88957143, 0.87288095, 0.8737381 , 0.89021429, 0.8737381 ,
       0.89021429, 0.87288095, 0.87288095, 0.88669048, 0.89021429,
       0.8737381 , 0.89021429, 0.88233333, 0.87404762, 0.8865    ,
       0.87288095, 0.8865    , 0.89021429, 0.8865    , 0.87288095,
       0.8857381 , 0.87288095, 0.87240476, 0.87288095, 0.89021429,
       0.8857381 , 0.87288095, 0.89021429, 0.87288095, 0.8865 

In [38]:
actions

[((2, 5, 300),
  {'learning_rate': 0.20909398034658444, 'reg_lambda': 0.7231212485077365}),
 ((4, 5, 400),
  {'learning_rate': 0.02506146596244337, 'reg_lambda': 0.37127688088633154}),
 ((4, 3, 100),
  {'learning_rate': 0.055279381066616826, 'reg_lambda': 0.3751903782737007}),
 ((2, 2, 400),
  {'learning_rate': 0.07550404987369105, 'reg_lambda': 0.632523646362735}),
 ((2, 3, 1000),
  {'learning_rate': 0.41517095203577126, 'reg_lambda': 0.6635365379871643}),
 ((4, 3, 1000),
  {'learning_rate': 0.0010570730338550984, 'reg_lambda': 0.30930924690552136}),
 ((3, 5, 100),
  {'learning_rate': 0.14003197793829403, 'reg_lambda': 0.6758587898648543}),
 ((3, 5, 300),
  {'learning_rate': 0.055279381066616826, 'reg_lambda': 0.3751903782737007}),
 ((3, 5, 400),
  {'learning_rate': 0.05710051788566583, 'reg_lambda': 0.3777489819421137}),
 ((4, 2, 300),
  {'learning_rate': 0.057099744231187716, 'reg_lambda': 0.37774789499731903}),
 ((4, 2, 200),
  {'learning_rate': 0.055279381066616826, 'reg_lambda': 

In [10]:
g(2, 1, 0.10178515794789737, 1.3023325726318398)

7.504816090819687

In [11]:
g(2, 1, 0.10178515794789737, 1)

9.77376644668603